<a href="https://colab.research.google.com/github/pravin691983/insaid-capstone_project_1/blob/development/SourceCode/insaid_capstone_project_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **INSAID - Capstone Project on Data Analytics**


# Problem Statement

**InsaidTelecom**, one of the leading telecom players, understands that customizing offering
is very important for its business to stay competitive.
Currently, InsaidTelecom is seeking to leverage behavioral data from more than 60% of the 50 million mobile devices active daily in India
to help its clients better understand and interact with their audiences.



# Objective

In this **consulting assignment**, Insaidians are expected to build a dashboard to understand
user's demographic characteristics based on their mobile usage, geolocation, and mobile device properties.
Doing so will help millions of developers and brand advertisers around the world pursue
data-driven marketing efforts which are relevant to their users and catered to their preferences.

To help the customer the consultants are expected to have depth of clarity in the underlying data.
How much effort has been put into cleansing and purifying the data will decide how closely have you looked at the data.
How detailed is the observation stated in the submission report and finally how well a group presents their consulting journey.

Please remember this is an analytics consulting hence, your efforts in terms of
finding user behavior is going to directly impact the company's offerings.
Do help the company understand what is the
right way forward and suggest actionable insights from marketing and product terms.

# Data Loading and Description

In this assignment, we are going to study the demographics of a user (gender and age) based on their app download and usage behaviors.
The Data is collected from mobile apps that use InsaidTelecom services. Full recognition and consent from individual user of those apps have been obtained,
and appropriate anonymization have been performed to protect privacy. Due to confidentiality, we won't provide details on how the gender and age data was obtained. <Br>


The data schema can be represented in the following table: <BR>

**gender_age_train** - Devices and their respective user gender, age and age_group

**phone_brand_device_model** - device ids, brand, and models phone_brand: note that few brands are in Chinese

// TODO :  added Table of devices list with brands

**events_data** - when a user uses mobile on INSAID Telecom network, the event gets logged in this data.
Each event has an event id, location (lat/long), and the event corresponds to frequency of mobile usage.
timestamp: when the user is using the mobile.

Download the DataSets for events_data from : [events_data.csv](https://https://drive.google.com/uc?export=download&id=1OK4yi-f1v03vS6fBMjuOLKPhNOfjzsh3)

Download the DataSets for **gender_age_train** & **phone_brand_device_model** onto Python by connecting to the below
provided MySQL instance.



---



***Importing a library that is not in Colaboratory*** 
<Br>
To import a library that's not in Colaboratory by default, you can use !pip install or !apt-get install.

In [1]:
!pip install -q mysql-connector-python-rf

     |████████████████████████████████| 11.9MB 343kB/s 


**Importing the Dataset**

In [0]:
import numpy as np                                                 # Implemennts milti-dimensional array and matrices
import pandas as pd                                                # For data manipulation and analysis
import pandas_profiling
import matplotlib.pyplot as plt                                    # Plotting library for Python programming language and it's numerical mathematics extension NumPy
import seaborn as sns                                              # Provides a high level interface for drawing attractive and informative statistical graphics

%matplotlib inline
sns.set()

from subprocess import check_output



---



In [0]:
#Import Required packages for MySQL instance
import mysql.connector
from mysql.connector import Error

**Utils functions to fetch data from MySQL**

Reference : https://dev.mysql.com/doc/connector-python/en/connector-python-examples.html

**Connecting to MySQL Using Connector/Python**

In [25]:
# Defining connection arguments in a dictionary and using the ** operator is another option:
config = {
  'user': 'student',
  'password': 'student',
  'host': 'cpanel.insaid.co',
  'database': 'Capstone1',
  'raise_on_warnings': True
}

# The connect() constructor creates a connection to the MySQL server and returns a MySQLConnection object.
try:
  cnx = mysql.connector.connect(**config)
except mysql.connector.Error as err:
  if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
    print("Something is wrong with your user name or password")
  elif err.errno == errorcode.ER_BAD_DB_ERROR:
    print("Database does not exist")
  else:
    print(err)
else:
  if cnx.is_connected():  
    db_Info = cnx.get_server_info()
    print("Connected to MySQL Server version ", db_Info)
    cursor = cnx.cursor()
    cursor.execute("select database();")
    record = cursor.fetchone()
    print("You're connected to database: ", record)


Connected to MySQL Server version  5.6.46-cll-lve
You're connected to database:  ('Capstone1',)


**Querying Data Using Connector/Python**

In [0]:
#Fetching data from MySQL(Capstone1) for gender_age_train
cursor.execute("select * from gender_age_train")
gender_age_train_data=cursor.fetchall()


In [27]:
#Converting data to DataFrame
gender_age_train_dataframe=pd.DataFrame(gender_age_train_data)
gender_age_train_dataframe.columns=['device_id','gender','age','age_group']
gender_age_train_dataframe.head()

,device_id,gender,age,age_group
0,-8076087639492063270,M,35,M32-38
1,-2897161552818060146,M,35,M32-38
2,-8260683887967679142,M,35,M32-38
3,-4938849341048082022,M,30,M29-31
4,245133531816851882,M,30,M29-31


In [0]:
#Fetching data from MySQL(Capstone1) for phone_brand_device_model
cursor.execute("select * from phone_brand_device_model")
phone_brand_device_model_data=cursor.fetchall()

In [29]:
#Converting data to DataFrame
phone_brand_device_model_dataframe=pd.DataFrame(phone_brand_device_model_data)
phone_brand_device_model_dataframe.columns=['device_id','brand','model']
phone_brand_device_model_dataframe.head()

,device_id,brand,model
0,1877775838486905855,vivo,Y13
1,-3766087376657242966,小米,V183
2,-6238937574958215831,OPPO,R7s
3,8973197758510677470,三星,A368t
4,-2015528097870762664,小米,红米Note2


**Closing connection after fetch data from DB**

In [0]:
# Closing connection after fetch data from DB
if cnx.is_connected():  
  cnx.close() 
  print("Closing database connection")



---



# Data Profiling


*   In the upcoming sections we will first understand our dataset using various pandas functionalities.
*   Then with the help of pandas profiling we will find which columns of our dataset need preprocessing.
*   In preprocessing we will deal with erronous and missing values of columns.
*   Again we will do pandas profiling to see how preprocessing have transformed our dataset.







---



**3.1 Understanding the Dataset**

To gain insights from data we must look into each aspect of it very carefully. We will start with observing few rows and columns of data both from the starting and from the end.

Let us check the basic information of the dataset. The very basic information to know is the dimension of the dataset – rows and columns – that’s what we find out with the method shape.

In [0]:
print("Understanding the Dataset")



---



**3.2 Pre Profiling**

By pandas profiling, **an interactive HTML report** gets generated which contins all the information about the columns of the dataset, like the counts and type of each column. Detailed information about each column, **correlation between different columns** and a sample of dataset.<Br>


*   It gives us **visual interpretation** of each column in the data.
*   Spread of the data can be better understood by the distribution plot.
*   Grannular level analysis of each column.

In [0]:
profile = pandas_profiling.ProfileReport(matches_data)
profile.to_file(outputfile="data_before_preprocessing.html")

Here, we have done Pandas Profiling before preprocessing our dataset, so we have named the html file as **data_before_preprocessing.html**. <Br>
Take a look at the file and see what useful insight you can develop from it.
Now we will process our data to better understand it.



---



**3.3 Preprocessing**

Here are the challenges that are persent in the Data:

1. There are nulls/zeros in "device_id" of events_data which should not be the case.
2. There are nulls/zeros in "latitude" and "longitude".
3. Some of the "latitudes" and "longitudes" are wrong.
4. "state" column of events_data has null values in the DataBase. Retrieve those rows and fill them appropriatly.



---



1. Device ID cannot be null.

	- Search for other columns that might have a similar bug.



In [0]:
print("Some precessing to fixed above issue")

2. The column "state" of "events_data" has null values.

In [0]:
print("Some precessing to fixed above issue")

3. Use folium package to plot latitudes and longitudes and see if there is any discrepancy in the position of some points. 

   If you find any anomaly, rectify them.

In [0]:
print("Some precessing to fixed above issue")



---



**3.4 Post Pandas Profiling**

Now we have preprocessed the data, now the dataset doesnot contain missing values. <BR>
You can compare the two reports, i.e **data_after_preprocessing.html** and **data_before_preprocessing.html**.


In **data_after_preprocessing.html** report, observations:

In the Dataset info, 
* Total Missing(%) = 0.0%
* Number of variables = 13



---



# Questions

**1. Distribution of Users(device_id) across States.**

**2. Distribution of Users across Phone Brands(Consider only 10 Most used Phone Brands).**

**3. Distribution of Users across Gender.**

**4. Distribution of Users across Age Segments.**

**5. Distribution of Phone Brands(Consider only 10 Most used Phone Brands) for each Age Segment, State, Gender.**

**6. Distribuiion of Gender for each State, Age Segment and Phone Brand(Consider only 10 Most used Phone Brands).**

**7. Distribution of Age Segments for each State, Gender and Phone Brand(Consider only 10 Most used Phone Brands).**

Note: While doing analysis for the above points, consider only one instance of a particular User(device_id) as a User can do multiple phone calls and considering every instance of the same User can give misleading numbers.

**8. Hourly distribution of Phone Calls.**

**9. Plot the Users on the Map using any suitable package.**

# Conclusion

// TODO :